### Imports

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, cv2
from google.colab import drive


### #Find an alternative approach to opening/uploading img file

In [58]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
image_folder = '/content/drive/MyDrive/My-imgs'

## Assign Lighting Score

In [60]:
def calc_weighted_sum(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l = lab[:, :, 0].astype(np.float32)/255
    perception = l.mean()
    shadow     = (l < 0.15).mean() #the avg l-values of the number of pixels w/l value < 15%
    highlight  = (l > 0.85).mean() #the avg l-values of the number of pixels w/l value > 85%
    score = perception - shadow + highlight*0.15
    return perception, shadow, highlight, score

print("Lighting score (P - (S+H)): ", calc_weighted_sum(img))


Lighting score (P - (S+H)):  (np.float32(0.6877265), np.float64(0.00020645112090714784), np.float64(0.05353273752152739), np.float64(0.6955499571574685))


### Try to make the score more accurate of the lighting and easier to work with

## Next: Create CSV file to include all images and their *scores*

In [63]:
import csv
data = [] #to-be a dictionary containing info to store in csv
def write_dict(filename):
      img = cv2.imread(os.path.join(image_folder, filename)) #reads the image from filename
      if img is not None:
        size = img.shape #(px, px, color channel)
        perception, shadow, highlight, score = calc_weighted_sum(img)
        data.append(
            {'Filename': filename, 'Size': size, 'Perception': perception, 'Shadow': shadow, 'Highlight': highlight, 'Score': score}
        )
      else:
        print(f"Warning: Could not read image file: {filename}")
      return data
for filename in os.listdir(image_folder): #iterates through each file in the drive folder
  write_dict(filename)
print(data)

[{'Filename': '20250615_141926.jpg', 'Size': (4000, 1848, 3), 'Perception': np.float32(0.5088547), 'Shadow': np.float64(0.047297348484848484), 'Highlight': np.float64(0.037229843073593075), 'Score': np.float64(0.4671418151900882)}, {'Filename': '20241031_210300.jpg', 'Size': (3392, 1908, 3), 'Perception': np.float32(0.15528789), 'Shadow': np.float64(0.684628834401329), 'Highlight': np.float64(4.8517167042442944e-05), 'Score': np.float64(-0.5293336651999146)}, {'Filename': '20240917_143309.jpg', 'Size': (2558, 2922, 3), 'Perception': np.float32(0.19142434), 'Shadow': np.float64(0.4545490546762074), 'Highlight': np.float64(0.003521986022832905), 'Score': np.float64(-0.2625964167630931)}, {'Filename': '20240907_142847.jpg', 'Size': (2992, 2516, 3), 'Perception': np.float32(0.3851754), 'Shadow': np.float64(0.17788254104214313), 'Highlight': np.float64(0.005191506975676526), 'Score': np.float64(0.20807159193703917)}, {'Filename': '20240907_184341.jpg', 'Size': (2992, 2992, 3), 'Perception':

In [64]:
with open('image-lightings.csv', 'w', newline='') as csvfile: #new csv file
    fieldnames = ['Filename', 'Size', 'Perception', 'Shadow', 'Highlight', 'Score'] #columns
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)

df = pd.read_csv('image-lightings.csv')
print(df.to_string()) #prints the csv file

               Filename             Size  Perception    Shadow  Highlight     Score
0   20250615_141926.jpg  (4000, 1848, 3)    0.508855  0.047297   0.037230  0.467142
1   20241031_210300.jpg  (3392, 1908, 3)    0.155288  0.684629   0.000049 -0.529334
2   20240917_143309.jpg  (2558, 2922, 3)    0.191424  0.454549   0.003522 -0.262596
3   20240907_142847.jpg  (2992, 2516, 3)    0.385175  0.177883   0.005192  0.208072
4   20240907_184341.jpg  (2992, 2992, 3)    0.254299  0.490072   0.015754 -0.233410
5   20250524_195551.jpg  (4000, 1848, 3)    0.416996  0.275634   0.095835  0.155737
6   20240928_223149.jpg  (2938, 2600, 3)    0.511496  0.036302   0.051147  0.482866
7   20240907_141844.jpg  (2992, 2992, 3)    0.505871  0.023945   0.073427  0.492940
8   20240907_094525.jpg  (2992, 2992, 3)    0.619224  0.033387   0.093592  0.599875
9   20241018_205719.jpg  (2992, 2992, 3)    0.196126  0.442139   0.006044 -0.245107
10  20250307_112143.jpg  (4000, 1848, 3)    0.461376  0.124402   0.034789  0